In [ ]:
import pyspark
import time
from pyspark.sql import SparkSession, SQLContext
from pyspark.sql.functions import *
import matplotlib.pyplot as plt

timing = []
core = (1, 2, 3, 4, 5, 6, 7, 8)

for x in core:
    spark_session = SparkSession.builder\
        .master("spark://192.168.2.122:7077") \
        .appName(f"Timing 1 workers {x} cores")\
        .config("spark.cores.max", f"{x}")\
        .config("spark.executor.memory", "2g")\
        .getOrCreate()

    sc = SQLContext(spark_session.sparkContext)

    start = time.time()

    df = spark_session.read.json("hdfs://130.238.28.211:9000/reddit/RC_2011-08.json")
    df.createOrReplaceTempView('park')
    df = sc.sql("select body,created_utc,subreddit from park").where(df["body"]!="[deleted]").withColumn("hour",hour(from_unixtime(col("created_utc")))).drop('created_utc')

    end = time.time()
    timing.append(end-start)
    print(f"cores={x}, elapsed_time={end-start:.3f} seconds")
    spark_session.stop()

### Plotting the results ###
plt.scatter(core, timing)

# Adding the perfect scaling line
one_core_time = timing[0]
perfect_scaling = [one_core_time / c for c in core]
plt.plot(core, perfect_scaling, linestyle='dashed', label='Perfect Scaling')

plt.xlabel('Cores')
plt.ylabel('Elapsed Time (seconds)')
plt.legend()
plt.savefig("new Timing Cores 1-8 For 1 Workers")